In [157]:
import os
import pandas as pd
import json
import pprint
import re
import nbformat

from collections import defaultdict

In [158]:
# Read the JSON file
with open("../DataProcessing/TestData/message.json", "r") as file:
    data = json.load(file)
    
with open("../DataProcessing/TestData/postGame.json", "r") as file:
    postData = json.load(file)
    
with open("../DataProcessing/TestData/runeSheet.json", "r") as file:
    runeSheet = json.load(file)
    
# Access the 'participants' array within 'metadata'
participantsData = data['metadata']['participants']

# Access the 'frames' array within 'metadata'
framesData = data['info']['frames']

participantFrames = [frame['participantFrames'] for frame in framesData]


# Create dictionaries to store the counts and most mentioned names for each ID
championCountsDict = defaultdict(lambda: defaultdict(int))
namesDict = defaultdict(list)

mostMentionedNamesDict = {}
participantData = []
participantChampionMapping = {}


In [159]:
targetPuuid = 'qqLzRit0Y8mcgNefznUaZUfIIny9LqJbFJ4tnRTdRDZ5rZliVKCJdK3DaEEKTtjU2MkSQTvGXDKarw'

In [160]:
# Function to calculate champion counts
def calculateChampionCounts(data):
    championCountsDict = {}
    namesDict = {}

    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    killerId = int(event.get('killerId'))
                    victimId = int(event.get('victimId'))

                    if killerId not in championCountsDict:
                        championCountsDict[killerId] = {}
                        namesDict[killerId] = []

                    if victimId not in championCountsDict:
                        championCountsDict[victimId] = {}
                        namesDict[victimId] = []

                    # Find the killer's and victim's participant data
                    killerNames = [victim.get('name') for victim in event.get('victimDamageReceived', [])]
                    victimNames = [killer.get('name') for killer in event.get('victimDamageDealt', [])]

                    # Use the last name mentioned as the killer name
                    if killerNames:
                        killerName = killerNames[-1]
                        if killerName not in championCountsDict[killerId]:
                            championCountsDict[killerId][killerName] = 1
                        else:
                            championCountsDict[killerId][killerName] += 1
                        namesDict[killerId].append(killerName)

                    # Use the last name mentioned as the victim name
                    if victimNames:
                        victimName = victimNames[-1]
                        if victimName not in championCountsDict[victimId]:
                            championCountsDict[victimId][victimName] = 1
                        else:
                            championCountsDict[victimId][victimName] += 1
                        namesDict[victimId].append(victimName)

    return championCountsDict, namesDict

def calculateMostMentionedChampionNames(data):
    # Calculate champion counts
    championCountsDict, namesDict = calculateChampionCounts(data)

    # Process the counts and choose the most mentioned champion name for each ID
    mostMentionedNamesDict = {}
    for participantId, championCounts in championCountsDict.items():
        mostMentionedChampion = max(championCounts, key=championCounts.get)
        mostMentionedNamesDict[participantId] = mostMentionedChampion
        count = championCounts[mostMentionedChampion]
        names = namesDict[participantId]

    return mostMentionedNamesDict

mostMentionedNamesDict = calculateMostMentionedChampionNames(data)

In [161]:
def addParticipantData(participantId, puuid, championName):  # add championName
    participantData.append([participantId, puuid, championName])  # add championName

def findParticipantData(data):
    if 'info' in data and 'participants' in data['info']:
        for participant in data['info']['participants']:
            participantId = participant['participantId']
            puuid = participant['puuid']
            
            championName = mostMentionedNamesDict[participantId]
            addParticipantData(participantId, puuid, championName)

findParticipantData(data)

In [162]:
def findParticipantByPuuid(participantData, targetPuuid):
    for entry in participantData:
        participantId, puuid, championName = entry
        if puuid == targetPuuid:
            return participantId, championName
    return None, None

# Example usage:
# Assuming 'targetPuuid' is the Puuid you want to find
targetParticipantId, targetChampionName = findParticipantByPuuid(participantData, targetPuuid)

if targetParticipantId is not None:
    print(f"Participant ID: {targetParticipantId}, Champion: {targetChampionName}")
else:
    print("Participant not found.")

Participant ID: 5, Champion: Senna


In [163]:
def excludeColumns(data, columnsToExclude=None, displayNoZero=False):
    # Exclude specified columns
    excludedData = {}

    for key, value in data.items():
        if key in columnsToExclude:
            continue  # Skip excluded columns
        elif isinstance(value, dict):
            # Recursively exclude columns within nested dictionaries
            excludedData[key] = excludeColumns(value, columnsToExclude, displayNoZero)
        else:
            excludedData[key] = value

    return excludedData

def displayNonZeroParams(participantFrame, displayNoZero=False, indent=''):
    # Iterate through the participant frame's items
    for key, value in participantFrame.items():
        if isinstance(value, dict):
            # If the value is a nested dictionary, iterate through its items
            print(f"{indent}{key} for {targetChampionName}: ")
            displayNonZeroParams(value, displayNoZero, indent + '  ')
        elif isinstance(value, (int, float)) and (not displayNoZero or value != 0):
            # Display the parameter if it's a numeric value and not zero
            print(f"{indent}  {key}: {value}")

In [164]:
def getParticipantDataAtMinute(data, participantId, targetMinute, columnsToExclude=None, displayNoZero=False):
    timestampData = []

    # Check if there are frames in the data
    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            timestamp = frame['timestamp']
            currentMinute = round(timestamp / 60000)  # Convert milliseconds to minutes and round to the nearest minute

            if currentMinute == targetMinute:
                participantFrames = frame.get('participantFrames', {})
                if participantId in participantFrames:
                    participantData = participantFrames[participantId]
                    timestampData.append([timestamp, participantData])

                    # Call the excludeColumns method
                    excludedData = excludeColumns(participantData, columnsToExclude, displayNoZero)

                    # Display either the original data or the excluded data based on your requirement
                    if displayNoZero:
                        print(f"Timestamp at minute {targetMinute}: {timestamp}")
                        displayNonZeroParams(excludedData, displayNoZero)
                        print("\n---\n")
                    else:
                        print(f"Timestamp at minute {targetMinute}: {timestamp}")
                        displayNonZeroParams(participantData, displayNoZero)
                        print("\n---\n")
    return timestampData

desiredParticipantId = (f"{targetParticipantId}")
desiredMinute = 2 
columnsToExclude = ['magicDamageDone', 'magicDamageDoneToChampions', 'magicDamageTaken', 
                    'physicalDamageDone', 'physicalDamageDoneToChampions', 'physicalDamageTaken', 
                    'trueDamageDone', 'trueDamageDoneToChampions', 'trueDamageTaken', 'timeEnemySpentControlled']

# Call the function with displayNoZero set to True
result = getParticipantDataAtMinute(data, desiredParticipantId, desiredMinute, columnsToExclude=columnsToExclude, displayNoZero=True)


Timestamp at minute 2: 120031
championStats for Senna: 
    armor: 34
    attackDamage: 56
    attackSpeed: 113
    health: 560
    healthMax: 560
    healthRegen: 8
    magicResist: 30
    movementSpeed: 330
    power: 350
    powerMax: 350
    powerRegen: 28
  currentGold: 32
damageStats for Senna: 
    totalDamageDone: 470
  goldPerSecond: 30
  level: 1
  participantId: 5
position for Senna: 
    x: 8217
    y: 1542
  totalGold: 532

---



In [165]:
def getKillDataForParticipant(data, participantId, participantData):
    killData = []

    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    killerId = int(event.get('killerId'))
                    victimId = int(event.get('victimId'))
                    assistIds = [int(assistId) for assistId in event.get('assistingParticipantIds', [])]

                    if killerId == participantId:
                        timestamp = event['timestamp']
                        victimDamageReceived = event.get('victimDamageReceived', [])

                        # Calculate total damage done by the killer to the victim
                        totalDamageDone = sum(
                            damageInfo['basic'] + damageInfo['magicDamage'] + damageInfo['physicalDamage'] + damageInfo['trueDamage']
                            for damageInfo in victimDamageReceived
                            if damageInfo['participantId'] == participantId
                        )

                        # Find names from participantData
                        killerName = next((participant[2] for participant in participantData if participant[0] == killerId),
                                           f"Unknown Champion with ID {killerId}")
                        victimName = next((participant[2] for participant in participantData if participant[0] == victimId),
                                           f"Unknown Champion with ID {victimId}")

                        # Find names of assisting champions and their damage
                        assistingChampions = []
                        damageByAssists = []

                        for assistId in assistIds:
                            assistName = next((participant[2] for participant in participantData if participant[0] == assistId),
                                               f"Unknown Champion with ID {assistId}")

                            assistDamage = sum(
                                damageInfo['basic'] + damageInfo['magicDamage'] + damageInfo['physicalDamage'] + damageInfo['trueDamage']
                                for damageInfo in victimDamageReceived
                                if damageInfo['participantId'] == assistId
                            )

                            assistingChampions.append(assistName)
                            damageByAssists.append(assistDamage)

                        killData.append({
                            'timestamp': timestamp,
                            'killerId': killerId,
                            'killerName': killerName,
                            'victimId': victimId,
                            'victimName': victimName,
                            'totalDamageDone': totalDamageDone,
                            'assistingChampions': assistingChampions,
                            'damageByAssists': damageByAssists
                        })

    return killData

# Example usage:
   # Replace with the desired participant ID
#targetParticipantId
killDataForParticipant = getKillDataForParticipant(data, targetParticipantId, participantData)

# Display the kill data with total damage done, assisting champions, and damage by assists
for kill in killDataForParticipant:
    assistingChampionsLine = f"Assisting Champions: {', '.join(kill['assistingChampions'])}.\n" if kill['assistingChampions'] else ""
    damageByAssistsLine = f"Damage Done by Assists: {', '.join([f'{damage} by {assist}' for damage, assist in zip(kill['damageByAssists'], kill['assistingChampions'])])}.\n" if kill['assistingChampions'] else ""

    print(f"Timestamp: {kill['timestamp']}\n"
          f"Killer: {kill['killerName']} (ID: {kill['killerId']})\n"
          f"Victim: {kill['victimName']} (ID: {kill['victimId']})\n"
          f"Total Damage Done: {kill['totalDamageDone']}\n"
          f"{assistingChampionsLine}"
          f"{damageByAssistsLine}")


Timestamp: 505464
Killer: Senna (ID: 5)
Victim: Thresh (ID: 10)
Total Damage Done: 1

Timestamp: 508357
Killer: Senna (ID: 5)
Victim: Vladimir (ID: 9)
Total Damage Done: 159

Timestamp: 653847
Killer: Senna (ID: 5)
Victim: Thresh (ID: 10)
Total Damage Done: 253
Assisting Champions: Caitlyn.
Damage Done by Assists: 396 by Caitlyn.

Timestamp: 1172851
Killer: Senna (ID: 5)
Victim: Rengar (ID: 7)
Total Damage Done: 615
Assisting Champions: Sion, MasterYi.
Damage Done by Assists: 513 by Sion, 1071 by MasterYi.

Timestamp: 1189438
Killer: Senna (ID: 5)
Victim: Vladimir (ID: 9)
Total Damage Done: 402
Assisting Champions: MasterYi, TwistedFate, Caitlyn.
Damage Done by Assists: 573 by MasterYi, 620 by TwistedFate, 1023 by Caitlyn.



In [166]:
def displayParticipantParameters(data, participantIds):

    # Display the extracted parameter values
    for participantId in participantIds:
        print(f"Participant {targetChampionName} Parameters:")
        for idx, frameData in enumerate(participantFrames):
            damageStats = frameData.get(participantId, {}).get('damageStats', {})
            totalDamageDone = damageStats.get('totalDamageDone', 0)
            totalDamageDoneToChampions = damageStats.get('totalDamageDoneToChampions', 0)
            totalDamageTaken = damageStats.get('totalDamageTaken', 0)

            print(f"Minute {idx + 1}:")
            print(f"  Total Damage Done: {totalDamageDone}")
            print(f"  Total Damage Done To Champions: {totalDamageDoneToChampions}")
            print(f"  Total Damage Taken: {totalDamageTaken}")

# Example usage:
# Assuming 'data' is the loaded JSON data
displayParticipantParameters(data, f"{targetParticipantId}")

Participant Senna Parameters:
Minute 1:
  Total Damage Done: 0
  Total Damage Done To Champions: 0
  Total Damage Taken: 0
Minute 2:
  Total Damage Done: 0
  Total Damage Done To Champions: 0
  Total Damage Taken: 0
Minute 3:
  Total Damage Done: 470
  Total Damage Done To Champions: 0
  Total Damage Taken: 0
Minute 4:
  Total Damage Done: 1086
  Total Damage Done To Champions: 58
  Total Damage Taken: 93
Minute 5:
  Total Damage Done: 1670
  Total Damage Done To Champions: 607
  Total Damage Taken: 746
Minute 6:
  Total Damage Done: 2014
  Total Damage Done To Champions: 700
  Total Damage Taken: 1339
Minute 7:
  Total Damage Done: 2758
  Total Damage Done To Champions: 700
  Total Damage Taken: 1339
Minute 8:
  Total Damage Done: 3847
  Total Damage Done To Champions: 1190
  Total Damage Taken: 1880
Minute 9:
  Total Damage Done: 5830
  Total Damage Done To Champions: 1419
  Total Damage Taken: 2288
Minute 10:
  Total Damage Done: 6271
  Total Damage Done To Champions: 1576
  Total D

In [167]:
def calculateDamageDifference(data, targetParticipantId, startFrame, endFrame, participantData):
    # Extract the numeric part of the targetParticipantId
    participantId = ''.join(filter(str.isdigit, targetParticipantId))

    # Determine the extra participantId based on the given participantId
    extraParticipantId = str((int(participantId) + 5) % 10) if int(participantId) < 6 else str((int(participantId) - 5) % 10)

    # Get the names of the participant and extra participant
    participantName = next((participant[2] for participant in participantData if participant[0] == int(participantId)),
                           f"Unknown Champion with ID {participantId}")

    extraParticipantName = next((participant[2] for participant in participantData if participant[0] == int(extraParticipantId)),
                                f"Unknown Champion with ID {extraParticipantId}")

    # Initialize variables to store total damage done to champions
    totalDamageToChampionsStart = 0
    totalDamageToChampionsEnd = 0
    totalDamageToChampionsExtraStart = 0
    totalDamageToChampionsExtraEnd = 0

    # Display the extracted parameter values and calculate total damage done to champions
    for idx, frameData in enumerate(participantFrames):
        damageStats = frameData.get(participantId, {}).get('damageStats', {})
        totalDamageDoneToChampions = damageStats.get('totalDamageDoneToChampions', 0)

        extraDamageStats = frameData.get(extraParticipantId, {}).get('damageStats', {})
        totalDamageDoneToChampionsExtra = extraDamageStats.get('totalDamageDoneToChampions', 0)

        if idx + 1 == startFrame:
            totalDamageToChampionsStart = totalDamageDoneToChampions
            totalDamageToChampionsExtraStart = totalDamageDoneToChampionsExtra
        elif idx + 1 == endFrame:
            totalDamageToChampionsEnd = totalDamageDoneToChampions
            totalDamageToChampionsExtraEnd = totalDamageDoneToChampionsExtra

    # Print the total damage done to champions for the participant and extra participant
    print(f"PRE")
    print(f"Minute {startFrame} - Participant {participantName}:")
    print(f"  Total Damage Done To Champions: {totalDamageToChampionsStart}")
    print(f"Minute {startFrame} - Extra Participant {extraParticipantName}:")
    print(f"  Total Damage Done To Champions: {totalDamageToChampionsExtraStart}")

    print(f"POST")
    print(f"Minute {endFrame} - Participant {participantName}:")
    print(f"  Total Damage Done To Champions: {totalDamageToChampionsEnd}")
    print(f"Minute {endFrame} - Extra Participant {extraParticipantName}:")
    print(f"  Total Damage Done To Champions: {totalDamageToChampionsExtraEnd}")

    # Calculate and print the differences
    difference = totalDamageToChampionsEnd - totalDamageToChampionsStart
    extraDifference = totalDamageToChampionsExtraEnd - totalDamageToChampionsExtraStart
    print(f"\nTotal Damage Done To Champions Difference (Participant {participantName}): {difference}")
    print(f"Total Damage Done To Champions Difference (Extra Participant {extraParticipantName}): {extraDifference}")

# Example usage:
# Assuming 'data' is the loaded JSON data and 'participantData' is the participant data
calculateDamageDifference(data, f'{targetParticipantId}', 5, 10, participantData)


PRE
Minute 5 - Participant Senna:
  Total Damage Done To Champions: 607
Minute 5 - Extra Participant Unknown Champion with ID 0:
  Total Damage Done To Champions: 0
POST
Minute 10 - Participant Senna:
  Total Damage Done To Champions: 1576
Minute 10 - Extra Participant Unknown Champion with ID 0:
  Total Damage Done To Champions: 0

Total Damage Done To Champions Difference (Participant Senna): 969
Total Damage Done To Champions Difference (Extra Participant Unknown Champion with ID 0): 0


In [168]:
def calculateDamageDifference(data, participantId, startFrame, endFrame):

    # Determine the extra participantId based on the given participantId
    extraParticipantId = str((int(participantId) + 5) % 10) if int(participantId) < 6 else str((int(participantId) - 5) % 10)

    extraParticipantName = next((participant[2] for participant in participantData if participant[0] == int(extraParticipantId)),
                                f"Unknown Champion with ID {extraParticipantId}")
    
    # Initialize variables to store total damage done parameters
    totalDamageDoneStart = 0
    totalDamageDoneEnd = 0
    totalDamageDoneExtraStart = 0
    totalDamageDoneExtraEnd = 0

    totalDamageDoneToChampionsStart = 0
    totalDamageDoneToChampionsEnd = 0
    totalDamageDoneToChampionsExtraStart = 0
    totalDamageDoneToChampionsExtraEnd = 0

    totalDamageTakenStart = 0
    totalDamageTakenEnd = 0
    totalDamageTakenExtraStart = 0
    totalDamageTakenExtraEnd = 0
    
    totalDamageTakenCompared = 0
    totalDamageDoneCompared = 0
    totalDamageDoneToChampionsCompared = 0

    # Display the extracted parameter values and calculate total damage done parameters
    for idx, frameData in enumerate(participantFrames):
        damageStats = frameData.get(participantId, {}).get('damageStats', {})
        totalDamageDone = damageStats.get('totalDamageDone', 0)
        totalDamageDoneToChampions = damageStats.get('totalDamageDoneToChampions', 0)
        totalDamageTaken = damageStats.get('totalDamageTaken', 0)

        extraDamageStats = frameData.get(extraParticipantId, {}).get('damageStats', {})
        totalDamageDoneExtra = extraDamageStats.get('totalDamageDone', 0)
        totalDamageDoneToChampionsExtra = extraDamageStats.get('totalDamageDoneToChampions', 0)
        totalDamageTakenExtra = extraDamageStats.get('totalDamageTaken', 0)

        if idx + 1 == startFrame:
            totalDamageDoneStart = totalDamageDone
            totalDamageDoneToChampionsStart = totalDamageDoneToChampions
            totalDamageTakenStart = totalDamageTaken
            totalDamageDoneExtraStart = totalDamageDoneExtra
            totalDamageDoneToChampionsExtraStart = totalDamageDoneToChampionsExtra
            totalDamageTakenExtraStart = totalDamageTakenExtra
        elif idx + 1 <= endFrame:
            totalDamageDoneEnd = totalDamageDone
            totalDamageDoneToChampionsEnd = totalDamageDoneToChampions
            totalDamageTakenEnd = totalDamageTaken
            totalDamageDoneExtraEnd = totalDamageDoneExtra
            totalDamageDoneToChampionsExtraEnd = totalDamageDoneToChampionsExtra
            totalDamageTakenExtraEnd = totalDamageTakenExtra
    
    # Print the pre and post data for the first participant
    print(f"Participant {targetChampionName}:")
    print(f"  Minute {startFrame} - Total Damage Done: {totalDamageDoneStart}")
    print(f"  Minute {endFrame} - Total Damage Done: {totalDamageDoneEnd}")
    print(f"  Minute {startFrame} - Total Damage Done To Champions: {totalDamageDoneToChampionsStart}")
    print(f"  Minute {endFrame} - Total Damage Done To Champions: {totalDamageDoneToChampionsEnd}")
    print(f"  Minute {startFrame} - Total Damage Taken: {totalDamageTakenStart}")
    print(f"  Minute {endFrame} - Total Damage Taken: {totalDamageTakenEnd}")

    # Print the data for the extra participant
    print(f"\nEnemy Laner: {extraParticipantName}")
    print(f"  Minute {startFrame} - Total Damage Done: {totalDamageDoneExtraStart}")
    print(f"  Minute {endFrame} - Total Damage Done: {totalDamageDoneExtraEnd}")
    print(f"  Minute {startFrame} - Total Damage Done To Champions: {totalDamageDoneToChampionsExtraStart}")
    print(f"  Minute {endFrame} - Total Damage Done To Champions: {totalDamageDoneToChampionsExtraEnd}")
    print(f"  Minute {startFrame} - Total Damage Taken: {totalDamageTakenExtraStart}")
    print(f"  Minute {endFrame} - Total Damage Taken: {totalDamageTakenExtraEnd}")

    # Calculate and print the differences
    damageDoneDifference = totalDamageDoneEnd - totalDamageDoneStart
    extraDamageDoneDifference = totalDamageDoneExtraEnd - totalDamageDoneExtraStart
    damageDoneToChampionsDifference = totalDamageDoneToChampionsEnd - totalDamageDoneToChampionsStart
    extraDamageDoneToChampionsDifference = totalDamageDoneToChampionsExtraEnd - totalDamageDoneToChampionsExtraStart
    damageTakenDifference = totalDamageTakenEnd - totalDamageTakenStart
    extraDamageTakenDifference = totalDamageTakenExtraEnd - totalDamageTakenExtraStart

    print(f"\nTotal Damage Done Difference ({targetChampionName}): {damageDoneDifference}")
    print(f"Total Damage Done Difference ({extraParticipantName}): {extraDamageDoneDifference}")
    print(f"Total Damage Done To Champions Difference ({targetChampionName}): {damageDoneToChampionsDifference}")
    print(f"Total Damage Done To Champions Difference ({extraParticipantName}): {extraDamageDoneToChampionsDifference}")
    print(f"Total Damage Taken Difference ({targetChampionName}): {damageTakenDifference}")
    print(f"Total Damage Taken Difference ({extraParticipantName}): {extraDamageTakenDifference}")
    
    # Calculate differences between participant and enemy
    totalDamageTakenCompared = damageTakenDifference - extraDamageTakenDifference
    totalDamageDoneCompared = damageDoneDifference - extraDamageDoneDifference
    totalDamageDoneToChampionsCompared = damageDoneToChampionsDifference - extraDamageDoneToChampionsDifference
    
    if totalDamageTakenCompared >= 0:
        print(f"\nParticipant {targetChampionName} took {totalDamageTakenCompared} more damage than enemy laner, ({extraParticipantName}) in given timeframe")
    else:
        print(f"\nParticipant {targetChampionName} took {abs(totalDamageTakenCompared)} less damage than enemy laner, ({extraParticipantName}) in given timeframe")

    if totalDamageDoneToChampionsCompared >= 0:
        print(f"\nParticipant {targetChampionName} beat enemy laner, ({extraParticipantName}) in given timeframe with total damage done difference of: {totalDamageDoneToChampionsCompared}")
    else:
        print(f"\nParticipant {targetChampionName} was beat by enemy laner, ({extraParticipantName}) in given timeframe with total damage done difference of: {abs(totalDamageDoneToChampionsCompared)}")

    if totalDamageDoneCompared >= 0:
        print(f"\nParticipant {targetChampionName} beat enemy laner, ({extraParticipantName}) in given timeframe with total damage done difference of: {totalDamageDoneCompared}")
    else:
        print(f"\nParticipant {targetChampionName} was beat by enemy laner, ({extraParticipantName}) in given timeframe with total damage done difference of: {abs(totalDamageDoneCompared)}")

# Example usage:
# Assuming 'data' is the loaded JSON data
calculateDamageDifference(data, f'{targetParticipantId}', 4, 10)


Participant Senna:
  Minute 4 - Total Damage Done: 1086
  Minute 10 - Total Damage Done: 6271
  Minute 4 - Total Damage Done To Champions: 58
  Minute 10 - Total Damage Done To Champions: 1576
  Minute 4 - Total Damage Taken: 93
  Minute 10 - Total Damage Taken: 3061

Enemy Laner: Unknown Champion with ID 0
  Minute 4 - Total Damage Done: 0
  Minute 10 - Total Damage Done: 0
  Minute 4 - Total Damage Done To Champions: 0
  Minute 10 - Total Damage Done To Champions: 0
  Minute 4 - Total Damage Taken: 0
  Minute 10 - Total Damage Taken: 0

Total Damage Done Difference (Senna): 5185
Total Damage Done Difference (Unknown Champion with ID 0): 0
Total Damage Done To Champions Difference (Senna): 1518
Total Damage Done To Champions Difference (Unknown Champion with ID 0): 0
Total Damage Taken Difference (Senna): 2968
Total Damage Taken Difference (Unknown Champion with ID 0): 0

Participant Senna took 2968 more damage than enemy laner, (Unknown Champion with ID 0) in given timeframe

Partici

In [169]:
def getVisionScoreAdvantageLaneOpponent(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            challengesData = participant.get('challenges', {})
            return challengesData.get('visionScoreAdvantageLaneOpponent', None)
    return None

def getVisionScore(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('visionScore', None)
    return None

def getKillParticipation(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('challenges', {}).get('killParticipation', None)
    return None

def getGoldPerMinute(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('challenges', {}).get('goldPerMinute', None)
    return None

def getTotalDamageDealtToChampions(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('totalDamageDealtToChampions', None)
    return None

def getTotalDamageTaken(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('totalDamageTaken', None)
    return None

def getControlWardTimeCoverageInRiverOrEnemyHalf(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('challenges', {}).get('controlWardTimeCoverageInRiverOrEnemyHalf', None)
    return None

def getControlWardsPlaced(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('challenges', {}).get('controlWardsPlaced', None)
    return None

def getSoloKills(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('challenges', {}).get('soloKills', None)
    return None

def getStealthWardsPlaced(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('challenges', {}).get('stealthWardsPlaced', None)
    return None

def getWardsKilled(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('wardsKilled', None)
    return None

def getWardsPlaced(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('wardsPlaced', None)
    return None

def getTeamBaronKills(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('challenges', {}).get('teamBaronKills', None)
    return None

def getTeamRiftHeraldKills(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('challenges', {}).get('teamRiftHeraldKills', None)
    return None

def getTeamElderDragonKills(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('challenges', {}).get('teamElderDragonKills', None)
    return None

def getDragonKills(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('dragonKills', None)
    return None

def getDeathsByEnemyChamps(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('challenges', {}).get('deathsByEnemyChamps', None)
    return None

def getDragonTakedowns(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('challenges', {}).get('dragonTakedowns', None)
    return None

def getKda(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('challenges', {}).get('kda', None)
    return None

def getTakedowns(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('challenges', {}).get('takedowns', None)
    return None

def getTurretTakedowns(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('challenges', {}).get('turretTakedowns', None)
    return None

def getLaneMinionsFirst10Minutes(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('challenges', {}).get('laneMinionsFirst10Minutes', None)
    return None

def getLaneRole(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('individualPosition', None)
    return None
        
def getJungleStats(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid and participant.get('individualPosition') == 'JUNGLE':
            totalAllyJungleMinionsKilled = participant.get('totalAllyJungleMinionsKilled', None)
            totalEnemyJungleMinionsKilled = participant.get('totalEnemyJungleMinionsKilled', None)
            return totalAllyJungleMinionsKilled, totalEnemyJungleMinionsKilled
    return None, None

def getGameWon(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            return participant.get('win', None)
    return None

laneRole = getLaneRole(postData, targetPuuid)
print(f"Role of {targetChampionName}: {laneRole}")

totalAllyJungleMinionsKilled, totalEnemyJungleMinionsKilled = getJungleStats(postData, targetPuuid)
if totalAllyJungleMinionsKilled is not None and totalEnemyJungleMinionsKilled is not None:
    print(f"Total Ally Jungle Minions Killed: {totalAllyJungleMinionsKilled}")
    print(f"Total Enemy Jungle Minions Killed: {totalEnemyJungleMinionsKilled}")

takedowns = getTakedowns(postData, targetPuuid)
print(f"TakeDowns by {targetChampionName}: {takedowns}")

soloKills = getSoloKills(postData, targetPuuid)
print(f"Solo Kills for {targetChampionName}: {soloKills}")

kda = getKda(postData, targetPuuid)
print(f"KDA by {targetChampionName}: {kda}")

killParticipation = getKillParticipation(postData, targetPuuid)
print(f"Kill Participation for {targetChampionName}: {killParticipation}\n")


totalDamageDealtToChampions = getTotalDamageDealtToChampions(postData, targetPuuid)
print(f"Total Damage Dealt To Champions by {targetChampionName}: {totalDamageDealtToChampions}")

totalDamageTaken = getTotalDamageTaken(postData, targetPuuid)
print(f"Total Damage Taken for {targetChampionName}: {totalDamageTaken}")

deathsByEnemyChamps = getDeathsByEnemyChamps(postData, targetPuuid)
print(f"Deaths by Enemy Champions on {targetChampionName}: {deathsByEnemyChamps}\n")


visionScore = getVisionScore(postData, targetPuuid)
print(f"\nVison Score for {targetChampionName}: {visionScore}")
    
visionScoreAdvantageLaneOpponent = getVisionScoreAdvantageLaneOpponent(postData, targetPuuid)
print(f"Vision Score Advantage on Lane Opponent for {targetChampionName}: {visionScoreAdvantageLaneOpponent}")

wardsPlaced = getWardsPlaced(postData, targetPuuid)
print(f"Wards Placed by {targetChampionName}: {wardsPlaced}")

controlWardTimeCoverageInRiverOrEnemyHalf = getControlWardTimeCoverageInRiverOrEnemyHalf(postData, targetPuuid)
print(f"Control Ward Time Coverage In River or Enemy Half for {targetChampionName}: {controlWardTimeCoverageInRiverOrEnemyHalf}")

controlWardsPlaced = getControlWardsPlaced(postData, targetPuuid)
print(f"Control Wards placed by {targetChampionName}: {controlWardsPlaced}")

stealthWardsPlaced = getStealthWardsPlaced(postData, targetPuuid)
print(f"Stealth Wards Placed by {targetChampionName}: {stealthWardsPlaced}")

wardsKilled = getWardsKilled(postData, targetPuuid)
print(f"Wards Killed by {targetChampionName}: {wardTakedowns}\n")


teamBaronKills = getTeamBaronKills(postData, targetPuuid)
print(f"Team Baron Kills by {targetChampionName}: {teamBaronKills}")

teamRiftHeraldKills = getTeamRiftHeraldKills(postData, targetPuuid)
print(f"Team Rift Herald Kills by {targetChampionName}: {teamRiftHeraldKills}")

teamElderDragonKills = getTeamElderDragonKills(postData, targetPuuid)
print(f"Team Elder Dragon Kills by {targetChampionName}: {teamElderDragonKills}")

dragonKills = getDragonKills(postData, targetPuuid)
print(f"Dragon Kills by {targetChampionName}: {dragonKills}")

dragonTakedowns = getDragonTakedowns(postData, targetPuuid)
print(f"Dragon Takedowns by {targetChampionName}: {dragonTakedowns}")

turretTakedowns = getTurretTakedowns(postData, targetPuuid)
print(f"Turret Takedowns by {targetChampionName}: {turretTakedowns}\n")


goldPerMinute = getGoldPerMinute(postData, targetPuuid)
print(f"Gold Per Minute for {targetChampionName}: {goldPerMinute}")

laneMinionsFirst10Minutes = getLaneMinionsFirst10Minutes(postData, targetPuuid)
if laneRole != "JUNGLE":
    print(f"Lane Minions First 10 Minutes by {targetChampionName}: {laneMinionsFirst10Minutes}\n")

gameWon = getGameWon(postData, targetPuuid)
if gameWon == True:
    print(f"{targetChampionName}'s Team Won the Game")
else:
    print(f"{targetChampionName}'s Team Lost the Game")

Role of Senna: UTILITY
TakeDowns by Senna: 19
Solo Kills for Senna: 2
KDA by Senna: 1.7272727272727273
Kill Participation for Senna: 0.40425531914893614

Total Damage Dealt To Champions by Senna: 16110
Total Damage Taken for Senna: 24231
Deaths by Enemy Champions on Senna: 11


Vison Score for Senna: 61
Vision Score Advantage on Lane Opponent for Senna: -0.19174116849899292
Wards Placed by Senna: 24
Control Ward Time Coverage In River or Enemy Half for Senna: None
Control Wards placed by Senna: 1
Stealth Wards Placed by Senna: 22
Wards Killed by Senna: 0

Team Baron Kills by Senna: 2
Team Rift Herald Kills by Senna: 1
Team Elder Dragon Kills by Senna: 0
Dragon Kills by Senna: 0
Dragon Takedowns by Senna: 2
Turret Takedowns by Senna: 0

Gold Per Minute for Senna: 361.310836791275
Lane Minions First 10 Minutes by Senna: 15

Senna's Team Won the Game
